In [8]:
# NDVI Analysis and Zoning for Silvopastures Using Shapefiles or Drone Data in Python

## 📦 1. Install Required Libraries (uncomment if needed)
# !pip install rasterio geopandas matplotlib numpy

import rasterio
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.mask import mask

## 🛰️ 2. Load the NIR (B8) and Red (B4) Bands from Sentinel-2 or drone images (GeoTIFF/JP2 converted to TIFF)

nir_path = "B08_10m.jp2"  # NIR band (842 nm)
red_path = "B04_10m.jp2"  # Red band (665 nm)

with rasterio.open(nir_path) as nir_src:
    nir = nir_src.read(1).astype("float32")
    profile = nir_src.profile

with rasterio.open(red_path) as red_src:
    red = red_src.read(1).astype("float32")

## 🌿 3. Calculate NDVI

ndvi = (nir - red) / (nir + red + 1e-6)
ndvi = np.clip(ndvi, -1, 1)  # Limit NDVI values to valid range

## 🗺️ 4. Save the NDVI as a new GeoTIFF

#ndvi_scaled = ((ndvi + 1) * 10000).astype('uint16')  # NDVI [-1,1] → [0, 20000]
#profile.update(dtype='uint16', count=1)

ndvi_path = "ndvi_silvopasture.tif"
#profile.update(dtype=rasterio.float32, count=1)

with rasterio.open("ndvi_silvopasture.tif", "w", **profile) as dst:
    dst.write(ndvi, 1)

## 5. Load a shapefile of silvopasture zones (if available)

#shapefile_path = "silvopasture_zones.shp"
#zones = gpd.read_file(shapefile_path)
#zones = zones.to_crs(profile["crs"])  # Ensure same CRS as raster

## 🔍 6. Mask NDVI by shapefile zones and extract statistics

def extract_stats_by_zone(ndvi_raster, gdf):
    results = []
    for i, row in gdf.iterrows():
        geom = [row["geometry"]]
        with rasterio.open(ndvi_raster) as src:
            out_image, out_transform = mask(src, geom, crop=True)
            data = out_image[0]
            data = data[data != src.nodata]
            mean_val = np.mean(data)
            results.append({
                "zone": row.get("name", f"Zone_{i}"),
                "mean_ndvi": mean_val
            })
    return results

zone_stats = extract_stats_by_zone(ndvi_path, zones)

## 📊 7. Show NDVI map and print statistics

plt.figure(figsize=(10, 6))
plt.title("NDVI for Silvopasture Area")
show(ndvi, cmap='RdYlGn')
plt.colorbar()
plt.show()

print("NDVI Zonal Statistics:")
for stat in zone_stats:
    print(stat)


NameError: name 'zones' is not defined